# Notebook 7: 链式法则与反向传播 - 责任如何分配

我们知道了最终的损失，也知道了梯度是降低损失的方向。但对于一个深度的、包含成千上万参数的神经网络，我们如何高效地计算出损失对每一个参数的偏导数呢？直接计算的复杂度是无法想象的。

答案就是反向传播（Backpropagation），而它的数学基石，就是微积分中的 **链式法则 (Chain Rule)**。

**目标:** 直观地理解链式法则，并明白反向传播是如何利用它，将最终的误差“传播”回网络中的每一个参数，从而高效计算梯度的。

**核心直觉:** 追责系统。
想象一个工厂的流水线，最终产品（神经网络的输出）出了问题（产生了损失）。为了改进，我们需要知道每个环节的工人（每个参数 `w` 和 `b`）对最终的问题负有多大的责任。

反向传播就像一个高效的“追责系统”。它从最终的问题开始，一层层地往回追溯，利用链式法则，精确地计算出每个工人（参数）对最终产品问题（损失）的“贡献度”（梯度）。

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 1. 链式法则的直觉

假设我们有一个简单的函数链： `z = f(y)` 且 `y = g(x)`。我们想知道 `x` 的微小变化如何影响 `z`，也就是求 `dz/dx`。

链式法则告诉我们： `dz/dx = dz/dy * dy/dx`

**直觉:** `x` 对 `z` 的总影响，等于 `y` 对 `z` 的影响，乘以 `x` 对 `y` 的影响。就像多米诺骨牌，第一个骨牌对第三个骨牌的影响，等于第二个对第三个的影响，乘以第一个对第二个的影响。

## 2. 一个简单的计算图

让我们用一个非常简单的例子来模拟神经网络的计算过程，并手动进行反向传播。

假设我们有输入 `x`，参数 `w` 和 `b`，计算过程如下：
1.  `u = w * x`
2.  `v = u + b`
3.  `L = v²` (我们的损失函数)

我们想知道损失 `L` 对参数 `w` 和 `b` 的偏导数，即 `∂L/∂w` 和 `∂L/∂b`。

In [2]:
# 假设一些具体值
x = 2
w = 3
b = 4

# --- 前向传播 ---
u = w * x       # u = 3 * 2 = 6
v = u + b       # v = 6 + 4 = 10
L = v**2        # L = 10^2 = 100

print(f"前向传播结果: u={u}, v={v}, L={L}")

前向传播结果: u=6, v=10, L=100


## 3. 手动反向传播

现在，我们从后往前，一步步计算偏导数。

**第1步：计算 `∂L/∂v`**
因为 `L = v²`，所以 `∂L/∂v = 2v`。
在我们的例子中，`v=10`，所以 `∂L/∂v = 2 * 10 = 20`。

**第2步：计算 `∂L/∂b` 和 `∂L/∂u`**
我们想求 `∂L/∂b`。根据链式法则，`∂L/∂b = (∂L/∂v) * (∂v/∂b)`。
- 我们已经知道 `∂L/∂v = 20`。
- 因为 `v = u + b`，所以 `∂v/∂b = 1`。
- 因此，`∂L/∂b = 20 * 1 = 20`。

同理，`∂L/∂u = (∂L/∂v) * (∂v/∂u)`。
- 因为 `v = u + b`，所以 `∂v/∂u = 1`。
- 因此，`∂L/∂u = 20 * 1 = 20`。

**第3步：计算 `∂L/∂w`**
我们想求 `∂L/∂w`。根据链式法则，`∂L/∂w = (∂L/∂u) * (∂u/∂w)`。
- 我们已经知道 `∂L/∂u = 20`。
- 因为 `u = w * x`，所以 `∂u/∂w = x`。
- 因此，`∂L/∂w = 20 * x = 20 * 2 = 40`。

**结论:** 我们成功地“反向传播”了误差，得到了损失对每个参数的梯度：`∂L/∂w = 40`，`∂L/∂b = 20`。

In [3]:
# --- 反向传播 ---

# 1. 计算 ∂L/∂v
dL_dv = 2 * v

# 2. 计算 ∂L/∂b 和 ∂L/∂u
dv_db = 1
dv_du = 1
dL_db = dL_dv * dv_db
dL_du = dL_dv * dv_du

# 3. 计算 ∂L/∂w
du_dw = x
dL_dw = dL_du * du_dw

print(f"计算出的梯度:")
print(f"∂L/∂w = {dL_dw}")
print(f"∂L/∂b = {dL_db}")

计算出的梯度:
∂L/∂w = 40
∂L/∂b = 20


## 4. 总结与展望

**关键回顾:**
1.  **链式法则是关键:** 它让我们能够计算嵌套函数的导数。
2.  **反向传播是算法:** 它是一种巧妙地应用链式法则的算法，从网络的末端开始，高效地计算出每一层参数的梯度。
3.  **先正向，再反向:** 必须先进行一次完整的前向传播来计算出每一环节的中间值，然后才能进行反向传播来计算梯度。

我们已经集齐了所有必要的工具：
- **前向传播** 来做预测。
- **损失函数** 来衡量预测的好坏。
- **反向传播** 来计算应该如何调整参数（梯度）。

在最后一个 Notebook 中，我们将把所有这些组合在一起，实现 **梯度下降 (Gradient Descent)** 算法，真正地让我们的网络“学习”起来！